In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2 as cv

In [2]:
training_dir = 'images/train'
testing_dir  = 'images/validation/'

In [3]:
imgSize = (150,150,3)

In [4]:

trainingSet = tf.keras.preprocessing.image_dataset_from_directory(training_dir,image_size=imgSize[:2],label_mode='categorical')
testingSet = tf.keras.preprocessing.image_dataset_from_directory(testing_dir,image_size=imgSize[:2],label_mode='categorical')

Found 28821 files belonging to 7 classes.
Found 7066 files belonging to 7 classes.


In [5]:
trainingSet.class_names

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten

In [7]:
model = Sequential()

model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255))

model.add(Conv2D(filters=32, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3,3),input_shape=(150,150,3), activation='relu',))
model.add(MaxPool2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(7,activation='softmax'))

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                metrics='accuracy')

In [8]:
model.fit(trainingSet,
             epochs=10,
             steps_per_epoch=len(trainingSet),
             validation_data=testingSet,
             validation_steps=len(testingSet))

Epoch 1/10
901/901 [==============================] - 371s 400ms/step - loss: 1.7706 - accuracy: 0.2771 - val_loss: 1.6194 - val_accuracy: 0.3624
Epoch 2/10
901/901 [==============================] - 143s 158ms/step - loss: 1.5151 - accuracy: 0.4156 - val_loss: 1.4491 - val_accuracy: 0.4495
Epoch 3/10
901/901 [==============================] - 142s 158ms/step - loss: 1.3552 - accuracy: 0.4849 - val_loss: 1.3662 - val_accuracy: 0.4837
Epoch 4/10
901/901 [==============================] - 142s 157ms/step - loss: 1.2552 - accuracy: 0.5274 - val_loss: 1.2955 - val_accuracy: 0.5161
Epoch 5/10
901/901 [==============================] - 142s 158ms/step - loss: 1.1827 - accuracy: 0.5536 - val_loss: 1.2679 - val_accuracy: 0.5342
Epoch 6/10
901/901 [==============================] - 142s 158ms/step - loss: 1.1248 - accuracy: 0.5773 - val_loss: 1.3043 - val_accuracy: 0.5095
Epoch 7/10
901/901 [==============================] - 142s 158ms/step - loss: 1.0690 - accuracy: 0.5967 - val_loss: 1.2668 -

In [9]:
model.save('trained weights 1.h5')

In [4]:
model = tf.keras.models.load_model("trained weights 1.h5")

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 150, 150, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0

In [15]:
face_cascade = cv.CascadeClassifier('haarcascade_frontalface_default.xml')

In [36]:
def predict_emotion(img):
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray,1.3,5)
            
    for (x,y,w,h) in faces:
        roi = np.expand_dims(cv.resize(frame[y:y+h, x:x+w], (150,150)),0)
        pred  = model.predict(roi)
        text_idx=np.argmax(pred)
        text_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']
        if text_idx == 0:
            text= text_list[0]
        if text_idx == 1:
            text= text_list[1]
        elif text_idx == 2:
            text= text_list[2]
        elif text_idx == 3:
            text= text_list[3]
        elif text_idx == 4:
            text= text_list[4]
        elif text_idx == 5:
            text= text_list[5]
        elif text_idx == 6:
            text= text_list[6]
        
        return(text)

In [38]:
cap = cv.VideoCapture(0)

while True:
    ret,frame = cap.read()
    if ret:
        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.3,5)
        for (x,y,w,h) in faces:
            text = predict_emotion(frame)
            cv.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),5)
            cv.putText(frame, text, (x, y-5),
                                cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255),2)
        cv.imshow("Face",frame)

    if cv.waitKey(10) == 27: break
cap.release()
cv.destroyAllWindows()
